In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

path = os.path.expanduser("~/Downloads/tim_data")

images = ["First", "Second", "Third", "Forth"]

emotions = ["Negative", "Neutral", "Positive"]

events = [31, 32, 33, 34,
          51, 52, 53, 54,
          71, 72, 73, 74]

columns = ["Event.Name", "Emotion", "Image", "SCR"]

measurement = "CDA.AmpSum"

all_subjects = pd.DataFrame(columns=columns)

averaged = pd.DataFrame(columns=["Emotion", "SCR"])

binned_df = pd.DataFrame(columns=["Emotion", "Bin", "SCR"])

In [ ]:
for file in os.listdir(path):
    if not os.path.isdir(os.path.join(path, file)):
        continue
    if ".DS_Store" in file:
        continue
    if not file.startswith("sub-AL") or file == 'sub-AL24':
        continue
    subject = file
    print(f"Processing subject {subject}")
    df = pd.read_csv(os.path.join(path, file, f"{subject}_era_4s.txt"), sep="\t")
    df = df[df["Event.Name"].isin(events)]

    df["Emotion"] = [emotions[int(name) // 20 - 1] for name in df["Event.Name"]]
    df["Image"] = [int(name) % 10 for name in df["Event.Name"]]
    df["SCR"] = df[measurement]

    all_subjects = pd.concat([all_subjects, df[columns]], ignore_index=True)

    averages = [{"Emotion": "Negative",
                "SCR": df[df["Event.Name"].isin([31, 32, 33, 34])][measurement].mean()},
                {"Emotion": "Neutral",
                "SCR": df[df["Event.Name"].isin([51, 52, 53, 54])][measurement].mean()},
                {"Emotion": "Positive",
                "SCR": df[df["Event.Name"].isin([71, 72, 73, 74])][measurement].mean()}]
    
    averaged = pd.concat([averaged, pd.DataFrame(averages)], ignore_index=True)

    df = pd.read_csv(os.path.join(path, file, f"{subject}_era_aggregated.txt"), sep="\t")
    df = df[df["Event.Name"].isin(events)]

    print(f"Example series - {df[(df['Event.Name'] == 31) & (df['Bin'] == 1)][measurement].values}")

    aveages = [{"Emotion": "Negative",
               "Bin": bin,
               "SCR": df[(df["Event.Name"] == 31) & (df["Bin"] == bin)][measurement].mean()} for bin in range(11)]
    aveages += [{"Emotion": "Neutral",
                "Bin": bin,
                "SCR": df[(df["Event.Name"] == 51) & (df["Bin"] == bin)][measurement].mean()} for bin in range(11)]
    aveages += [{"Emotion": "Positive",
                "Bin": bin,
                "SCR": df[(df["Event.Name"] == 71) & (df["Bin"] == bin)][measurement].mean()} for bin in range(11)]
    
    binned_df = pd.concat([binned_df, pd.DataFrame(aveages)], ignore_index=True)

In [ ]:
import seaborn as sns

color_map = {
        'Negative': 'red',
        'Neutral': 'orange',
        'Positive': 'green'
    }


sns.catplot(data=all_subjects, x="Image", y="SCR", hue="Emotion", errorbar="se", capsize=.2, kind="point", palette=color_map)
plt.title(f"{measurement} Response - Not Averaged")
plt.xlabel("Image Number")
plt.ylabel("SCR")

In [ ]:
sns.histplot(data=all_subjects, x="SCR", hue="Emotion", palette=color_map, multiple="stack")
plt.title(f"{measurement} Response - Not Averaged")
plt.xlabel("SCR Amplitude")
plt.ylabel("Count")

In [ ]:
sns.barplot(data=all_subjects, x="Emotion", y="SCR", palette=color_map, errorbar="se")
plt.title(f"{measurement} by Emotion - Not Averaged")

In [ ]:
sns.histplot(data=averaged, x="SCR", hue="Emotion", palette=color_map, multiple="stack")
plt.title(f"{measurement} Response - Averaged")
plt.xlabel("SCR Amplitude")
plt.ylabel("Count")

In [ ]:
sns.barplot(data=averaged, x="Emotion", y="SCR", palette=color_map, errorbar="se")
plt.title(f"{measurement} by Emotion - Averaged")

In [ ]:
from scipy import stats

negative = averaged[averaged["Emotion"] == "Negative"]["SCR"]
neutral = averaged[averaged["Emotion"] == "Neutral"]["SCR"]
positive = averaged[averaged["Emotion"] == "Positive"]["SCR"]

t_stat, p_val = stats.ttest_ind(negative, neutral)
print(f"Negative vs Neutral: t-statistic = {t_stat}, p-value = {p_val}")
t_stat, p_val = stats.ttest_ind(negative, positive)
print(f"Negative vs Positive: t-statistic = {t_stat}, p-value = {p_val}")
t_stat, p_val = stats.ttest_ind(neutral, positive)
print(f"Neutral vs Positive: t-statistic = {t_stat}, p-value = {p_val}")

In [ ]:
import seaborn as sns

sns.catplot(data=binned_df, x="Bin", y="SCR", hue="Emotion", errorbar="se", capsize=.2, kind="point", palette=color_map)
plt.title(f"{measurement} Response per Bin - Averaged")
plt.xlabel("Image Number")
plt.ylabel("SCR")